In [1]:
from __future__ import division
import sys
sys.path.insert(0, "~/.local/lib/python3.6/site-packages")

import sympy
from sympy import *
from sympy.physics.quantum import *

def express(a, b, name):
    sym = symbols(name)
    sol = solve(a-sym, b)
    assert len(sol) == 1
    return (sym, sol[0])


def frange(x, y, jump):
    i = 0
    curr = x + i*jump
    
    while curr < y:
        yield curr
        i += 1
        curr = x + i*jump
    

import gmpy2
import mpmath
gmpy2.get_context().precision = 10
mpmath.mp.dps = 10

import numpy
import scipy.linalg

In [2]:
# https://stackoverflow.com/q/59523322/1137334

from sympy.core.operations import AssocOp

def apply_ccr(expr, ccr, reverse=False):
    if not isinstance(expr, Basic):
        raise TypeError("The expression to simplify is not a sympy expression.")
        
    if not isinstance(ccr, Eq):
        if isinstance(ccr, Basic):
            ccr = Eq(ccr, 0)
        else:
            raise TypeError("The canonical commutation relation is not a sympy expression.")
    
    comm = None
    
    for node in preorder_traversal(ccr):
        if isinstance(node, Commutator):
            comm = node
            break
            
    if comm is None:
        raise ValueError("The cannonical commutation relation doesn not include a commutator.")
        
    solutions = solve(ccr, comm)
    
    if len(solutions) != 1:
        raise ValueError("There are more solutions to the cannonical commutation relation.")
        
    value = solutions[0]
    
    A = comm.args[0]
    B = comm.args[1]
    
    if reverse:
        (A, B) = (B, A)
        value = -value
    
    def is_expandable_pow_of(base, expr):
        return isinstance(expr, Pow) \
            and base == expr.args[0] \
            and isinstance(expr.args[1], Number) \
            and expr.args[1] >= 1
    
    
    def walk_tree(expr):
        if isinstance(expr, Number):
            return expr
        
        if not isinstance(expr, AssocOp) and not isinstance(expr, Function):
            return expr.copy()
        
        elif not isinstance(expr, Mul):
            return expr.func(*(walk_tree(node) for node in expr.args))
        
        else:
            args = [arg for arg in expr.args]
            
            for i in range(len(args)-1):
                x = args[i]
                y = args[i+1]
                
                if B == x and A == y:
                    args = args[0:i] + [A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if B == x and is_expandable_pow_of(A, y):
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and A == y:
                    xpow = Pow(B, x.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
                
                if is_expandable_pow_of(B, x) and is_expandable_pow_of(A, y):
                    xpow = Pow(B, x.args[1] - 1)
                    ypow = Pow(A, y.args[1] - 1)
                    args = args[0:i] + [xpow, A*B - value, ypow] + args[i+2:]
                    return walk_tree( Mul(*args).expand() )
            
            return expr.copy()
            
    
    return walk_tree(expr)
   

Basic.apply_ccr = lambda self, ccr, reverse=False: apply_ccr(self, ccr, reverse)


In [82]:
hbar = symbols('hbar', real=True, positive=True)

eye = Matrix.eye(2)

s1_ = \
    Matrix([
        [0,1],
        [1,0]
    ])
    
s2_ = \
    Matrix([
        [0,-I],
        [I,0,]
    ])

s3_ = Matrix([
        [1,0],
        [0,-1]
    ])

sm1_, sm2_, sm3_ = [TensorProduct(hbar/2 * s_, eye) for s_ in (s1_,s2_,s3_)]
se1_, se2_, se3_ = [TensorProduct(eye, hbar/2 * s_) for s_ in (s1_,s2_,s3_)]


In [83]:
mr, c, alpha = symbols('m_r, c, alpha')
A = symbols('A', positive=True)
D = symbols('D', negative=True)

E0 = -mr * c**2 * alpha**2 / 2
Eye = Matrix.eye(4)
E0_ = e0*Eye

smse_ = (sm1_*se1_ + sm2_*se2_ + sm3_*se3_)

H_ = E0_  +  A/hbar**2 * smse_  +  D/hbar**2 * sm3_*se3_
H_

Matrix([
[A/4 + D/4 - alpha**2*c**2*m_r/2,                                0,                                0,                               0],
[                              0, -A/4 - D/4 - alpha**2*c**2*m_r/2,                              A/2,                               0],
[                              0,                              A/2, -A/4 - D/4 - alpha**2*c**2*m_r/2,                               0],
[                              0,                                0,                                0, A/4 + D/4 - alpha**2*c**2*m_r/2]])

In [84]:
print(latex(H_*4))

\left[\begin{matrix}A + D - 2 \alpha^{2} c^{2} m_{r} & 0 & 0 & 0\\0 & - A - D - 2 \alpha^{2} c^{2} m_{r} & 2 A & 0\\0 & 2 A & - A - D - 2 \alpha^{2} c^{2} m_{r} & 0\\0 & 0 & 0 & A + D - 2 \alpha^{2} c^{2} m_{r}\end{matrix}\right]


In [85]:
print(latex(E0))

- \frac{\alpha^{2} c^{2} m_{r}}{2}


In [86]:
smse_

Matrix([
[hbar**2/4,          0,          0,         0],
[        0, -hbar**2/4,  hbar**2/2,         0],
[        0,  hbar**2/2, -hbar**2/4,         0],
[        0,          0,          0, hbar**2/4]])

In [87]:
print(latex(smse_*4/hbar**2))

\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 2 & 0\\0 & 2 & -1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]


In [89]:
sm3_*se3_

Matrix([
[hbar**2/4,          0,          0,         0],
[        0, -hbar**2/4,          0,         0],
[        0,          0, -hbar**2/4,         0],
[        0,          0,          0, hbar**2/4]])

In [90]:
print(latex(sm3_*se3_*4/hbar**2))

\left[\begin{matrix}1 & 0 & 0 & 0\\0 & -1 & 0 & 0\\0 & 0 & -1 & 0\\0 & 0 & 0 & 1\end{matrix}\right]


In [91]:
HP_inv, HD_ = H_.diagonalize()
HP_ = HP_inv.inv()

HP_ = Matrix([ (HP_.col(j) / HP_.col(j).norm()).T for j in range(4) ]).T
HP_inv = HP_.inv()

assert (HP_inv* HD_ * HP_ - H_).is_zero


In [92]:
HP_

Matrix([
[0, -sqrt(2)/2, sqrt(2)/2, 0],
[0,  sqrt(2)/2, sqrt(2)/2, 0],
[1,          0,         0, 0],
[0,          0,         0, 1]])

In [93]:
print(latex(HP_))

\left[\begin{matrix}0 & - \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} & 0\\0 & \frac{\sqrt{2}}{2} & \frac{\sqrt{2}}{2} & 0\\1 & 0 & 0 & 0\\0 & 0 & 0 & 1\end{matrix}\right]


In [94]:
HD_

Matrix([
[-3*A/4 - D/4 - alpha**2*c**2*m_r/2,                               0,                               0,                               0],
[                                 0, A/4 - D/4 - alpha**2*c**2*m_r/2,                               0,                               0],
[                                 0,                               0, A/4 + D/4 - alpha**2*c**2*m_r/2,                               0],
[                                 0,                               0,                               0, A/4 + D/4 - alpha**2*c**2*m_r/2]])

In [95]:
print(latex(HD_))

\left[\begin{matrix}- \frac{3 A}{4} - \frac{D}{4} - \frac{\alpha^{2} c^{2} m_{r}}{2} & 0 & 0 & 0\\0 & \frac{A}{4} - \frac{D}{4} - \frac{\alpha^{2} c^{2} m_{r}}{2} & 0 & 0\\0 & 0 & \frac{A}{4} + \frac{D}{4} - \frac{\alpha^{2} c^{2} m_{r}}{2} & 0\\0 & 0 & 0 & \frac{A}{4} + \frac{D}{4} - \frac{\alpha^{2} c^{2} m_{r}}{2}\end{matrix}\right]


In [98]:
print(latex(s3_))

\left[\begin{matrix}1 & 0\\0 & -1\end{matrix}\right]
